# Building with AutoGen 
### Please Complete the **[environment setup](part1_environment_setup.ipynb)** first before progressing with the notebook!


## 1.Starting Building with AutoGen 
Let's start with a simple example of how AutoGen is integrated Azure OpenAI. 
The example will confirm Azure OpenAI endpoints and keys are correctly configured and that AutoGen is installed and working correctly.


In [12]:
import os 
import asyncio
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

# Load environment variables from .env
load_dotenv()

# Create a minimal Azure OpenAI client.
azure_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Create an assistant agent with a simple system message.
agent = AssistantAgent(
    name="azure_test_agent",
    model_client=azure_client,
    system_message="You are a helpful Azure test assistant."
)

async def test_connection():
    user_input = "Hello! Are you working?"
    await Console(agent.run_stream(task=user_input))

await test_connection()


---------- user ----------
Hello! Are you working?
---------- azure_test_agent ----------
Hello! Yes, I'm here to help. How can I assist you today?


The **autogen** framework provides multiple agent roles—such as `AssistantAgent`, `UserAgent`, `FunctionAgent` (for function or tool calls), and `MultiAgent` (for orchestrating multiple participants)—each designed for specialized behaviors in a conversation. For example:

- `AssistantAgent`: Represents a helpful AI assistant.
- `FunctionAgent`: Handles specific tools or API calls.
- `MultiAgent`: Coordinates interactions among different agents.

This modular design makes it straightforward to build and manage multi-turn, multi-role dialogues, assigning clear, distinct responsibilities to each agent.


`AssistantAgent` we used here is a high-level wrapper around a language model client (such as an AzureOpenAI model) that manages:

1. **Conversational Context**: It keeps track of all previous exchanges and organizes them into a coherent dialog.
2. **System or Instruction Prompt**: You can give it a “system message” that defines the overarching instructions or persona, such as:
   > "You are a helpful Azure test assistant."

By consolidating these elements, `AssistantAgent` simplifies the process of building chat interfaces on top of LLMs, handling how user messages and model responses flow, and ensuring all conversation logic is neatly contained in a single agent object.


# 1.2 Function Calling Example 

Function calling in the context of modern LLMs (Large Language Models) lets the model “call” a function (or tool) whenever it detects the user’s query needs that function’s capabilities. Rather than returning a text-only answer, the LLM returns structured arguments for the function, which your application or framework executes. \

In AutoGen’s AssistantAgent, every function you register is treated as a “tool” that the LLM can invoke. When a user prompt suggests one of those tools is needed, the model produces a structured “function call” specifying which tool to use and the arguments to pass. The AssistantAgent runs that function in your Python environment, then feeds the result back to the model. Finally, the model incorporates that real-world data—whether weather forecasts, currency exchanges, or local times—into its final, user-facing response. This design keeps things neat: the LLM decides what needs doing, and your Python code handles how it’s done.

In [33]:
import os
import asyncio
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

# 1) Define three different "tool" functions that your LLM might call.

async def get_weather(city: str) -> str:
    """Returns a fake weather report for demonstration."""
    return f"The current weather in {city} is 25°C, sunny."

async def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Returns a simple stub result for currency conversion."""
    # In real scenarios, you'd call a currency conversion API here.
    # We'll just do a mock 1:1.1 ratio for demonstration.
    conversion_rate = 1.1
    converted_amount = amount * conversion_rate
    return f"{amount} {from_currency.upper()} is approximately {converted_amount:.2f} {to_currency.upper()}."

async def get_time_in(tz: str) -> str:
    """Returns a mock local time for the given timezone."""
    # You might call a real API or Python library like pytz/dateutil in production.
    return f"The current local time in {tz} is 09:00 AM."

# 2) Load environment variables from .env
load_dotenv()

# 3) Create the minimal Azure OpenAI client.
azure_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# 4) Create the AssistantAgent, giving it access to the three tools.
#    We mention them in the system_message so the model knows it can call them.
agent = AssistantAgent(
    name="multi_tool_agent",
    model_client=azure_client,
    tools=[get_weather, convert_currency, get_time_in],
    system_message=(
        "You are a helpful Azure test assistant. You have access to the following tools:\n"
        "1) get_weather(city: str) - Provides a weather report.\n"
        "2) convert_currency(amount: float, from_currency: str, to_currency: str) - Converts between currencies.\n"
        "3) get_time_in(tz: str) - Returns local time in a given timezone.\n\n"
        "Only call these functions if the user is requesting relevant info."
    )
)

# 5) Let’s define an async function that prompts for multiple tasks.
async def demo_multi_tool_calls():
    # We'll ask a series of user queries that might trigger each function separately.
    user_prompts = [
        "Hi there! Could you tell me the weather in Berlin?",
        "Actually, could you also convert 100 USD to EUR?",
        "What's the local time in Tokyo right now?"
    ]

    for prompt in user_prompts:
        print(f"---------- user ----------\n{prompt}")
        # Using run_stream for streaming the conversation to the console
        await Console(agent.run_stream(task=prompt))
        print("\n")

# 6) Run the demonstration in Jupyter by simply 'await'-ing the function.
await demo_multi_tool_calls()





---------- user ----------
Hi there! Could you tell me the weather in Berlin?
---------- user ----------
Hi there! Could you tell me the weather in Berlin?


---------- multi_tool_agent ----------
[FunctionCall(id='call_KYmYDQCeWctdFOeoQSuQNHKq', arguments='{"city":"Berlin"}', name='get_weather')]
---------- multi_tool_agent ----------
[FunctionExecutionResult(content='The current weather in Berlin is 25°C, sunny.', call_id='call_KYmYDQCeWctdFOeoQSuQNHKq', is_error=False)]
---------- multi_tool_agent ----------
The current weather in Berlin is 25°C, sunny.


---------- user ----------
Actually, could you also convert 100 USD to EUR?
---------- user ----------
Actually, could you also convert 100 USD to EUR?
---------- multi_tool_agent ----------
[FunctionCall(id='call_DCJBY8F8nPwCFIhwFVY31ckb', arguments='{"amount":100,"from_currency":"USD","to_currency":"EUR"}', name='convert_currency')]
---------- multi_tool_agent ----------
[FunctionExecutionResult(content='100.0 USD is approximately 110.00 EUR.', call_id='call_DCJBY8F8nPwCFIhwFVY31ckb', is_error=False)]
---------- multi_tool_agent ----------
100.0 USD is approximately 110.00 EUR.


----

### 1.2.1 Function Calling with Structured Outputs  

In [ ]:
import os
import asyncio
import nest_asyncio
from typing import Optional
from pydantic import BaseModel
from dotenv import load_dotenv

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool  # ✅ Correct import

# 1) Apply nest_asyncio (fixes Jupyter asyncio issues)
nest_asyncio.apply()

# 2) Pydantic model for structured outputs.
class QueryResponse(BaseModel):
    weather_report: Optional[str] = None
    currency_info: Optional[str] = None
    local_time: Optional[str] = None
    notes: Optional[str] = None

# 3) Updated function definitions (sync versions).
def get_weather(city: str) -> str:
    """Returns a fake weather report."""
    return f"The current weather in {city} is 25°C, sunny."

def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Returns a stub currency conversion."""
    rate = 1.1
    result = amount * rate
    return f"{amount} {from_currency.upper()} is ~{result:.2f} {to_currency.upper()}."

def get_time_in(tz: str) -> str:
    """Returns a mock local time."""
    return f"The current local time in {tz} is 09:00 AM."

# 4) Convert each function into FunctionTool for structured calling.
tool_get_weather = FunctionTool(get_weather, description="Get the current weather for a city.")
tool_convert_currency = FunctionTool(convert_currency, description="Convert an amount between currencies.")
tool_get_time_in = FunctionTool(get_time_in, description="Get the current time in a timezone.")

# 5) Load env variables from .env
load_dotenv()

# 6) Create a minimal Azure OpenAI client.
azure_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

# 7) Create an AssistantAgent with FunctionTool + enforced JSON output.
agent = AssistantAgent(
    name="multi_tool_agent",
    model_client=azure_client,
    tools=[tool_get_weather, tool_convert_currency, tool_get_time_in],
    system_message=(
        "You are a helpful Azure test assistant. You have access to:\n"
        "1) get_weather(city: str) - Fetches current weather details.\n"
        "2) convert_currency(amount: float, from_currency: str, to_currency: str) - Converts currency values.\n"
        "3) get_time_in(tz: str) - Retrieves the local time for a given timezone.\n\n"
        "⚠️ **STRICT OUTPUT FORMAT REQUIRED:**\n"
        "Your response MUST be in **valid JSON** formatted as follows:\n"
        "{\n"
        '  "weather_report": string or null,\n'
        '  "currency_info": string or null,\n'
        '  "local_time": string or null,\n'
        '  "notes": string or null\n'
        "}\n"
        "Ensure that your response **ONLY contains JSON**. No extra text or explanations."
    )
)

# 8) Demo function that prompts multiple tasks.
async def demo_multi_tool_calls():
    user_prompts = [
        "Hi there! Could you tell me the weather in Berlin?",
        "Please convert 100 USD to EUR.",
        "What's the local time in Tokyo right now?"
    ]
    for prompt in user_prompts:
        print(f"---------- user ----------\n{prompt}")
        await Console(agent.run_stream(task=prompt))
        print()

# 9) Execute the function in a Jupyter cell.
await demo_multi_tool_calls()


---------- user ----------
Hi there! Could you tell me the weather in Berlin?
---------- user ----------
Hi there! Could you tell me the weather in Berlin?
---------- multi_tool_agent ----------
[FunctionCall(id='call_TZRIhIUgzPs1O3QiCsiM1DZR', arguments='{"city":"Berlin"}', name='get_weather')]
---------- multi_tool_agent ----------
[FunctionExecutionResult(content='The current weather in Berlin is 25°C, sunny.', call_id='call_TZRIhIUgzPs1O3QiCsiM1DZR', is_error=False)]
---------- multi_tool_agent ----------
The current weather in Berlin is 25°C, sunny.

---------- user ----------
Please convert 100 USD to EUR.
---------- user ----------
Please convert 100 USD to EUR.


Compared to the previous iteration, which simply allowed an LLM-powered assistant to call basic functions, the updated code enhances reliability, structure, and execution flow. The original version used standard Python functions as tools, but lacked strict enforcement of structured outputs, meaning responses could be inconsistent or require additional post-processing. In contrast, the improved version wraps functions using FunctionTool from autogen_core.tools, ensuring proper argument parsing and structured execution. 